# 09 추천 시스템
## 08 파이썬 추천 시스템 패키지 - Surprise
+ R은 recommenderlabm,Spark는 MLib에서 쉽게 Recommendation을 수행할 수 있는 패키지를 가지고 있는 반면에 사이킷런에는 Recommendation을 쉽게 수행할 수 있는 package를 가지고 있지 않다.
+ python에서 recommendation을 쉽게 제공하는 대표적인 패키지로서 surprise가 있다. surprise는 scikit learn의 API와 유사하게 작성되어 있으며 이를 이용해 recommendation process를 쉽게 적용할 수 있다.
+ pip 또는 conda로 설치할 수 있으며, 윈도우 운영체제에 설치시에는 visual studio build tools이 미리 설치되어 있어야 한다.

#### surprise 패키지를 이용한 추천 수행 프로세스

+ 데이터 로딩
  + 데이터 컬럼 (무조건 userid, itemid, rating 순서로 데이터 정렬)
  + DataFrame에서 데이터 로딩
+ 모델 설정 및 학습
  + 추천 Algorithm 설정 (SVD, KNNBasic 등)
  +  Train 데이터로 학습 (train() 메소드)
+ 예측 및 평가
  + 예측 (test(): 전체 데이터 / predict(): 한개의 데이터)
  + 평가 (accuracy.rmse 등)

#### Surprise를 이용한 추천 구현 기본
1. 필요한 라이브러리 로딩
    + from surprise import SVD, Dataset, accuracy
    + from surprise.model_selection import train_test_split
2. 필요한 데이터 세트를 로딩. 데이터는 Dataset 패키지를 이용. Csv 파일 및 pandas dataframe 에서도 loading 가능. 로딩한 데이터 세트를 학습용과 테스트용 데이터 세트로 분리
   + data = Dataset.load_buitin('ml-100k')
   + trainset, testset = train_test_split(data, test_size=.25)
3. 행렬 분해를 수행할 알고리즘으로 SVD 생성하고 학습용 데이터로 학습
    + algo = SVD()
    + algo.fit(trainset)
4. 테스트 데이터 세트에 대해서 prediction을 수행. 일반적인 scikit learn의 predict() 메소드는 surprise에서 test() 메소드 특정 사용자와 item에 대한 predict는 predict() 메소드
    + predictions = algo.test(testset)

#### Surprise 주요 모듈 소개 - Dataset
+ Surprise는 무비 렌즈 데이터 세트와 같이 userid, itemid, rating 컬럼들이 사용자(userid)를 기준으로 한 로우 레벨의 평점 데이터로 구성된 데이터 세트에만 입력 가능하다.
+ 입력받은 데이터의 첫 번째 칼럼을 사용자 ID, 두번째 칼럼을 item ID, 세번째 칼럼을 Rating으로 가정한다. 네번째 부터는 recommendation 알고리즘에 아예 사용하지 않는다.
+ 이렇게 로우 레벨로 입력 받은 사용자-아이템 데이터는 Dataset 객체로 로딩 후 사용자-아이템 평점 행렬로 변환된다.

#### Dataset 클래스의 주요 메소드
메소드명 | 설명
--------|-------------
Dataset.load_buitin(name='ml-100k') | 무비렌즈 아카이브 FTP 서버에서 무비렌즈 데이터를 내려받는다. </br> ml-100k, ml-1M을 내려받을 수 있다. 일단 내려받은 데이터는 .surprise_data 디렉터리 밑에 저장되고, 해딩 디렉터리에 데이터가 있으면 FTP에서 내려받지 않고 해당 데이터를 이용한다. </br> 입력 파라미터인 name으로 대상 데이터가 ml-100k인지 ml-1m인지를 입력한다.(name='ml-100k)</br> 디폴트는 ml-100k이다.
Dataset.load_from_file(file_path, reader) |  os 파일에서 데이터를 로딩할 때 사용한다. </br> 콤마, 탭 등으로 컬럼이 분리된 포맷의 os 파일에서 데이터를 로딩한다. </br> 입력 파라미터로 os 파일 명, reader로 파일의 포맷을 지정한다.
Dataset.load_from_df(df, reader) | 판다스의 DataFrame에서 데이터를 로딩한다. </br> 파라미터로 DataFrame을 입력받으며 DataFrame 역시 반드시 3개의 컬럼인 사용자 아이디, 아이템 아이디, 평점 순으로 컬럼 순서가 정해져 있어야 한다. </br> 입력 파라미터로 DataFrame 객체, Reader로 파일의 포맷을 지정한다.

#### Surprise 주요 모듈 소개 - Reader
+ Raw 데이터 소스에서 Dataset로 로딩 규칙을 지정하기 위해 사용된다.
+ Surprise 데이터 세트는 기본적으로 무비렌즈 데이터와 같은 로우 레벨의 사용자 아이템 평점 데이터 형식을 따르므로 무비렌즈 데이터 형식이 아닌 경우 이를 변환하여 Dataset로 로딩해야 한다.
> from surprise import Reader
> reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
> data=Dataset.load_from_file('./ml-latest-small/rating_noh.csv', reader=reader) 
+ line_format (string) : 컬럼을 순서대로 나열한다. 입력된 문자열을 공백으로 분리해 컬럼으로 인식한다.
+ sep (char) : 컬럼을 분리하는 분리자이며, 디폴트는 '\t'이다. 판다스 DataFrame에서 입력받을 경우에는 기재할 필요가 없다.
+ rating_scale ( tuple, optional ): 평점 값의 최소 ~ 최대 평점을 설정하낟. 디폴트는 (1, 5)이지만 ratings.csv 파일의 경우는 최소 평점이 0.5, 최대 평점이 5이므로 (0.5, 5)로 설정했다.